In [7]:
# 名前など記入してください
Student_id = 200968
Name = "松山夏樹(MATSUYAMA Natsuki)"
Department = "sys"

In [8]:
# Traveling Salesman Problem using Branch and Bound
# Given the cost matrix of N cities (adj[i][j] means the cost of traveling from city i to city j), please find the  minimal cost to visit every city exactly once and return to the starting city.
# Starting point can be any city.
# Print out the minimum cost and the path you've found in following format.
# Minimum cost: 35
# Path: 0->2->4->1->3->0 

# import
import math 

In [9]:
# Define and initialize variables
# adj: adjacency matrix for the given graph 
# N: number of cities/nodes
# final_path: stores the final path of the salesman
# visited: keeps track of the already visited nodes
# final_res: stores the minimum weight of shortest path sofar

adj = [[0, 21, 5, 15, 9], 
       [17, 0, 12, 6, 24], 
       [13, 5, 0, 20, 8], 
       [9, 12, 7, 0, 23],
       [26, 7, 13, 8, 0]] 
N = 5
  
final_path = [None] * (N + 1)  
visited = [False] * N 
final_res = float('inf') 

In [11]:

# 現在の道筋を最後の道筋にコピーする 
def copyToFinal(curr_path): 
    final_path[:N + 1] = curr_path[:] # final_pathの0,...,N番目はcurr_pathのN個
    final_path[N] = curr_path[0] # final_pathの最後N番目をcurr_pathの0番目とする
  
#　頂点iでの最小値を見つける
def firstMin(adj, i): 
    min = float('inf')
    for k in range(N): 
        if adj[i][k] < min and i != k: #自分へ移動は除く
            min = adj[i][k] # 行列adjでi行での最小値を見つける
  
    return min
  
# 頂点iでの2番目の値を見つける
def secondMin(adj, i): 
    first, second = float('inf'), float('inf')
    for j in range(N): 
        if i == j: 
            continue
        if adj[i][j] <= first: 
            second = first 
            first = adj[i][j] 
  
        elif (adj[i][j] <= second and adj[i][j] != first): 
            second = adj[i][j] # 行列adjでi行での2番目の最小値を見つける
  
    return second 
  
# function that takes as arguments: 
# curr_bound -> lower bound of the root node 
# curr_weight-> stores the weight of the path so far 
# level-> current level while moving 
# in the search space tree 
# curr_path[] -> where the solution is being stored 
# which would later be copied to final_path[] 
def TSPRec(adj,curr_bound,curr_weight,level,curr_path,visited): 
    global final_res #関数を実行した結果,final_resが書き換えられる
      
    # 全てのノードを1度ずつは通り、levelがNであることを前提にする
    if level == N: 
          
        # 最初の頂点に向かうpathの最後の頂点から出るedgeがあるか確認する
        if adj[curr_path[level - 1]][curr_path[0]] != 0: 
              
            # curr_resでは通った道のweightの和が記録されている 
            curr_res = curr_weight + adj[curr_path[level - 1]][curr_path[0]] 
            if curr_res < final_res: 
                copyToFinal(curr_path) 
                final_res = curr_res 
        return
  
    # level = Nでなければ、すべての頂点を反復して検索スペースツリーを再帰的に構築する 
    for i in range(N): 
          
        # 次の頂点が同じでない場合は考慮する
        # 隣接行列の斜めのエントリで、まだ訪れていない場合 
        if (adj[curr_path[level-1]][i] != 0 and visited[i] == False): 
            temp = curr_bound # tempにcurr_boundを保存
            curr_weight += adj[curr_path[level - 1]][i] # curr_weightに通るノードの重みを足す
  
            # レベル2のcurr_boundの計算が他のレベルと異なる場合
            if level == 1: 
                curr_bound -= ((firstMin(adj,curr_path[level - 1]) +firstMin(adj, i))/2) 
            else: 
                curr_bound -= ((secondMin(adj,curr_path[level - 1]) +firstMin(adj, i))/2) 
  
            # curr_bound + curr_weight が今まで訪れたノードの中で実際のlower bound   
            # 現在のlower boundがfinal_resより小さければ、まだノードを探索する必要がある 
            
            if curr_bound + curr_weight < final_res: 
                curr_path[level] = i 
                visited[i] = True
                  
                # levelを+1し、次のlevelでの探索を行う
                TSPRec(adj,curr_bound,curr_weight,level + 1,curr_path,visited) 
  
            # そうでない場合は、curr_weightとcurr_boundへのすべての変更をリセットしてノードを剪定する
            curr_weight -= adj[curr_path[level - 1]][i] #1手戻す
            curr_bound = temp #1手戻す temp = curr_bound
  
            # visited arrayもリセットする
            visited = [False] * len(visited) #リセット
            for j in range(level): 
                if curr_path[j] != -1: 
                    visited[curr_path[j]] = True
  
# final_pathを設定する 
def TSP(adj): 
      
    # root nodeへの最初のlower boundを計算する
    # 公式:1/2 * (sum of first min + second min)を全ての頂点に適用 
    # curr_path と visited array を初期化
    curr_bound = 0
    curr_path = [-1] * (N + 1) 
    visited = [False] * N 
  
    # 最初のboundを計算する
    for i in range(N): 
        curr_bound += (firstMin(adj,i) + secondMin(adj,i)) 
  
    # lower boundを整数に丸める
    curr_bound = math.ceil(curr_bound/2) 
  
    # 頂点1から初める(curr_path[]の1番目は0であるため)
    visited[0] = True
    curr_path[0] = 0
  
    # TSPRecをcurr_weightのために呼び出す 
    # 0とレベル1に等しい
    TSPRec(adj,curr_bound,0,1,curr_path,visited) 
  

In [12]:
# Find the path based on given adjacent matrix
# Print out the result
TSP(adj) 
  
print("Minimum cost :",final_res) 
print("Path: ", end = ' ') 
for i in range(N): 
    print(final_path[i] + 1,end = '->') 
print(final_path[N] + 1)

Minimum cost : 35
Path:  1->3->5->2->4->1
